# Чтение и грубая очистка данных

In [1]:
import pandas as pd
import numpy as np
import string, re
from collections import Counter

from src.data.read_raw_data import read_from_gsheet, drop_unwanted_data, split_types

In [2]:
# Читаем данные из таблицы, удаляем неинтересующие столбцы, фильтруем по типу задания
raw_data = read_from_gsheet()
data = drop_unwanted_data(raw_data)
_, email_data = split_types(data)

C:\Users\max_a\AppData\Local\Temp\ipykernel_15316\2553523661.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  raw_data = read_from_gsheet()


In [3]:
 # Далее будем работать с данными по письмам
email_data.head()

,Type,Question id,Question,Text,Solving a communicative task,Text structure,Use of English (for emails)
2,Email,,…I am so happy that summer has come and we are...,"Dear Ronny, I was glad to hear from you again....",1.0,2.0,2.0
3,Email,,…I am learning to cook from my mother now. But...,"Dear Mary, Thanks for your message. It was gre...",1.0,2.0,2.0
4,Email,,…I am so happy that summer has come and we are...,Moscow\n15 october\nHi!\nThanks you for you re...,1.0,0.0,0.0
5,Email,,…I’ve recently been involved in a school surve...,"Hey, Mike.\r\nHow's it going? As for me, i'm p...",0.0,0.0,0.0
6,Email,,…All of my friends think camping is a perfect ...,"Hi Emily,\r\nThank you for the e-mail. I'm so ...",2.0,1.0,0.0


In [5]:
email_data = email_data[email_data['Text'].str.strip().astype(bool)] # Удалим строки с пустыми ответами

In [6]:
# К сожалению, пока придется удалить и строки, в которых задание кратко описано на русском или отсутствует
email_data.drop(email_data[email_data['Question'].str.contains('[А-Яа-я]+')].index, inplace=True)

# Очистка и токенизация текста

In [7]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

import contractions

from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\max_a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\max_a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
email_data['Text'] = email_data['Text'].str.lower() # Приведем символы к нижнему регистру
email_data['Text'] = email_data['Text'].apply(lambda x: contractions.fix(x)) # Преобразуем сокращения в полную запись: e.g. I'll -> I will
email_data['Text'] = email_data['Text'].apply(lambda x: re.sub('[\.\n\r\t]', ' ', x)) # Уберем символы переноса строк, табуляции
email_data['Text'] = email_data['Text'].apply(lambda x: re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-’]+', " ", x)) # Удалим символы пунктуации

In [9]:
# Добавим колонку с токенизированным текстом (показалось, что правильнее будет токенизировать тексты по отдельности)
email_data['Tokenized'] = email_data['Text'].apply(word_tokenize)

In [ ]:
# Возможно, НЕ стоит очищать текст от stopwords, так как при этом, например, мы теряем конструкции со вспомогательным глаголом
# filtered_tokens = [word for word in tokens if not word in stopwords.words('english')]

In [138]:
tokens = list()
for tokens_list in email_data['Tokenized']:
    tokens.extend(tokens_list)
bigrams = ngrams(tokens, 3)
cnt_bi = Counter(bigrams)

print(*cnt_bi.most_common(200))

(('by', 'the', 'way'), 300) (('you', 'asked', 'me'), 271) (('asked', 'me', 'about'), 255) (('the', 'way', 'tell'), 240) (('way', 'tell', 'me'), 239) (('tell', 'me', 'more'), 228) (('me', 'more', 'about'), 228) (('for', 'your', 'recent'), 225) (('from', 'you', 'in'), 221) (('you', 'in', 'your'), 218) (('i', 'am', 'always'), 206) (('messages', 'from', 'you'), 192) (('more', 'about', 'your'), 190) (('get', 'messages', 'from'), 186) (('to', 'get', 'messages'), 185) (('in', 'your', 'email'), 181) (('thanks', 'for', 'your'), 179) (('your', 'email', 'you'), 176) (('drop', 'me', 'a'), 173) (('me', 'a', 'line'), 173) (('all', 'for', 'now'), 170) (('that', 'is', 'all'), 162) (('is', 'all', 'for'), 160) (('your', 'recent', 'email'), 157) (('email', 'you', 'asked'), 156) (('soon', 'best', 'wishes'), 146) (('now', 'drop', 'me'), 144) (('email', 'i', 'am'), 142) (('a', 'line', 'best'), 137) (('line', 'best', 'wishes'), 137) (('recent', 'email', 'i'), 136) (('thank', 'you', 'for'), 135) (('you', 'for

In [10]:
# Соберем токены обратно в текст
email_data['prep_text'] = [' '.join(map(str, l)) for l in email_data['Tokenized']]

# Кодирование текста

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [23]:
features = ['Question', 'prep_text']
targets = ['Solving a communicative task', 'Text structure', 'Use of English (for emails)']

test_size = int(0.25 * email_data.shape[0])  # Количество документов в тестовой выборке

# Разделим на тестовые и тренировочные данные
test_data_index = email_data.sample(test_size).index
train_data_index = email_data.index.difference(test_data_index)
test_data = email_data.loc[test_data_index]
train_data = email_data.loc[train_data_index]

In [36]:
def vectorize_as_bongrams(texts: list[str], ngram_min=1, ngram_max=2, max_words=3000):
    """
    Кодирует список текстов в список из закодированных векторов с помощью Bag of N-grams
    :param texts: Список текстов для кодирования
    :param ngram_min: Минимальная кратность n-граммы
    :param ngram_max: Максимальная кратность n-граммы
    :param max_words: Максимальное количество слов в словаре (размерность признакового пространства)
    :return: 
    """
    count_vectorizer = CountVectorizer(ngram_range=(ngram_min, ngram_max), max_features=max_words)
    emb = count_vectorizer.fit_transform(texts).toarray()
    print("Создан vectorizer с размерностью признаков:", str(np.array(emb).shape[1]))
    return emb, count_vectorizer

In [41]:
train_encoded, train_vectorizer = vectorize_as_bongrams(train_data['prep_text'].to_list())
test_encoded = train_vectorizer.transform(test_data['prep_text'].to_list()).toarray() 

Создан vectorizer с размерностью признаков: 3000


In [60]:
def print_out(emb, feat, ngram):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names_out()[:25], "\n")
    print(ngram,"bag-of-feature: ")
    print(feat.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(emb.transpose(), index=feat.get_feature_names_out()).head(), "\n")

In [61]:
print_out(train_encoded, train_vectorizer, ngram='1-2-грамма')

1-2-грамма bag-of-words: 
['able' 'able to' 'about' 'about arguments' 'about dreams' 'about family'
 'about friends' 'about friendship' 'about grandparents' 'about holidays'
 'about horse' 'about household' 'about how' 'about it' 'about languages'
 'about making' 'about me' 'about my' 'about school' 'about that'
 'about the' 'about their' 'about this' 'about to' 'about travelling'] 

1-2-грамма bag-of-feature: 
{'dear': 576, 'ronny': 2043, 'was': 2675, 'glad': 931, 'to': 2487, 'hear': 1057, 'from': 886, 'you': 2920, 'again': 51, 'actually': 36, 'summer': 2246, 'in': 1186, 'russia': 2051, 'is': 1239, 'quite': 1985, 'hot': 1138, 'and': 128, 'though': 2464, 'it': 1279, 'depends': 604, 'on': 1804, 'certain': 435, 'region': 2023, 'be': 280, 'honest': 1128, 'my': 1621, 'favourite': 787, 'season': 2074, 'winter': 2825, 'because': 300, 'this': 2455, 'time': 2470, 'with': 2855, 'new': 1696, 'year': 2914, 'if': 1169, 'parents': 1865, 'me': 1506, 'visit': 2656, 'would': 2893, 'happy': 1003, 'spen